# Lab 3 - Ray and Dask

## Creating an inverted index and Word Counting

Please review Lab 2 before proceeding. Part of this lab is creating an inverted index, but using Ray instead of Parallel. We'll then move onto the more complicated word counting example.

In [1]:
import ray
ray.init(ignore_reinit_error=True)

2023-10-15 17:51:33,766	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-10-15 17:51:35,566	INFO worker.py:1642 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.7.1


In [2]:
%load_ext autoreload
%autoreload 2


# Put all your solutions into Lab1_helper.py as this script which is autograded
import Lab3_helper
    
import os
from pathlib import Path
home = str(Path.home())

import pandas as pd

## Inverted Index

**Exercise 1:**

You have already written most of what you need to use Ray to construct distributed inverted indices. Here I want you to modify Lab3_helper.py to use Ray and return the final inverted index. I'm supplying the code that divides your books into three sets.

In [3]:
group1 = Lab3_helper.get_book_files(f"{home}/csc-369-student/data/gutenberg/group1")
group2 = Lab3_helper.get_book_files(f"{home}/csc-369-student/data/gutenberg/group2")
group3 = Lab3_helper.get_book_files(f"{home}/csc-369-student/data/gutenberg/group3")

In [4]:
index = Lab3_helper.merge([group1,group2,group3])

In [5]:
index['Education']

{'/home/paul/csc-369-student/data/gutenberg/group2/3600-0.txt': [438645,
  549522,
  3030373,
  3030389],
 '/home/paul/csc-369-student/data/gutenberg/group2/45-0.txt': [420655],
 '/home/paul/csc-369-student/data/gutenberg/group2/408-0.txt': [141091,
  142312,
  142732],
 '/home/paul/csc-369-student/data/gutenberg/group3/1399-0.txt': [1285944],
 '/home/paul/csc-369-student/data/gutenberg/group3/2814-0.txt': [4070],
 '/home/paul/csc-369-student/data/gutenberg/group3/766-0.txt': [351347]}

In [6]:
index['Education']

{'/home/paul/csc-369-student/data/gutenberg/group2/3600-0.txt': [438645,
  549522,
  3030373,
  3030389],
 '/home/paul/csc-369-student/data/gutenberg/group2/45-0.txt': [420655],
 '/home/paul/csc-369-student/data/gutenberg/group2/408-0.txt': [141091,
  142312,
  142732],
 '/home/paul/csc-369-student/data/gutenberg/group3/1399-0.txt': [1285944],
 '/home/paul/csc-369-student/data/gutenberg/group3/2814-0.txt': [4070],
 '/home/paul/csc-369-student/data/gutenberg/group3/766-0.txt': [351347]}

In [7]:
# clean up memory to help us all co-exist on the same machine
index = None
import gc
gc.collect()

0

## Word Counting
Now consider a different problem of common interest. Suppose we have a large corpus (fancy word common in natural language processing) and we want to calculate the number of times a word appears. We could try to hack our inverted index, but let's insert the requirement that this must be a clean implementation. In other words, I'll be manually reviewing your design and making you redo the assignment if it isn't "clean". 

**Exercise 2:**

Write a function that counts the words in a book. Output format shown below. You do not have to worry about punctuation and capitalization. In other words, please stick to simple f.readlines() and line.split(" "). Do not strip anything out.

In [8]:
counts = Lab3_helper.count_words(group1[0])

In [9]:
counts

{'\ufeffThe': 1,
 'Project': 66,
 'Gutenberg': 19,
 'EBook': 1,
 'of': 567,
 'Alice’s': 13,
 'Adventures': 5,
 'in': 381,
 'Wonderland,': 3,
 'by': 75,
 'Lewis': 4,
 'Carroll\n': 4,
 '\n': 960,
 'This': 19,
 'eBook': 4,
 'is': 84,
 'for': 138,
 'the': 1546,
 'use': 23,
 'anyone': 3,
 'anywhere': 3,
 'United': 12,
 'States': 7,
 'and': 734,
 'most\n': 1,
 'other': 36,
 'parts': 3,
 'world': 8,
 'at': 191,
 'no': 66,
 'cost': 3,
 'with': 206,
 'almost': 8,
 'restrictions\n': 1,
 'whatsoever.': 2,
 '': 664,
 'You': 18,
 'may': 24,
 'copy': 8,
 'it,': 39,
 'give': 11,
 'it': 339,
 'away': 16,
 'or': 124,
 're-use': 2,
 'under': 20,
 'terms': 19,
 'of\n': 37,
 'License': 8,
 'included': 2,
 'this': 112,
 'online': 4,
 'at\n': 18,
 'www.gutenberg.org.': 2,
 'If': 18,
 'you': 287,
 'are': 53,
 'not': 119,
 'located': 7,
 'States,': 3,
 "you'll": 2,
 'have\n': 1,
 'to': 734,
 'check': 4,
 'laws': 9,
 'country': 5,
 'where': 12,
 'before': 22,
 'using': 8,
 'ebook.\n': 2,
 'Title:': 1,
 'Wonder

**Exercise 3**

Now let's distribute this using Ray. Please implement a function that parallelizes the word counting and subsequent merges.

In [10]:
merged_counts = Lab3_helper.merge_count_words([group1,group2,group3])

In [11]:
merged_counts

{'\ufeffThe': 44,
 'Project': 4855,
 'Gutenberg': 1405,
 'EBook': 88,
 'of': 234481,
 'Alice’s': 16,
 'Adventures': 29,
 'in': 131087,
 'Wonderland,': 4,
 'by': 35942,
 'Lewis': 11,
 'Carroll\n': 4,
 '\n': 203089,
 'This': 5111,
 'eBook': 344,
 'is': 55261,
 'for': 58736,
 'the': 416040,
 'use': 2396,
 'anyone': 844,
 'anywhere': 302,
 'United': 851,
 'States': 490,
 'and': 267326,
 'most\n': 919,
 'other': 9606,
 'parts': 546,
 'world': 2195,
 'at': 49078,
 'no': 19631,
 'cost': 473,
 'with': 70689,
 'almost': 3221,
 'restrictions\n': 36,
 'whatsoever.': 152,
 '': 277065,
 'You': 5326,
 'may': 8645,
 'copy': 667,
 'it,': 8354,
 'give': 5099,
 'it': 63715,
 'away': 4528,
 'or': 29702,
 're-use': 142,
 'under': 5130,
 'terms': 1663,
 'of\n': 18919,
 'License': 592,
 'included': 169,
 'this': 29698,
 'online': 296,
 'at\n': 3545,
 'www.gutenberg.org.': 83,
 'If': 4463,
 'you': 53664,
 'are': 24032,
 'not': 53558,
 'located': 435,
 'States,': 223,
 "you'll": 121,
 'have\n': 3250,
 'to': 2

In [12]:
merged_counts['things']

3499

In [13]:
# Don't forget to push!